In [1]:
# Importing modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
from pyspark.sql import SparkSession
import findspark
import joblib
from sqlalchemy import create_engine
from sklearn.linear_model import LogisticRegression




In [2]:
engine = create_engine('sqlite:///../Resources/Data/diabetes_health_indicators.db') 
query = "SELECT * FROM diabetes_indicators;" 
diabetes_data_df = pd.read_sql(query, engine)

In [3]:
diabetes_data_df

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,2.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [4]:
#cleaned_diabetes_data = diabetes_data_df.drop(columns=['GenHlth','MentHlth','PhysHlth','CholCheck','AnyHealthcare','NoDocbcCost','Education','Income'])


In [5]:
#Separating Data

y = diabetes_data_df['Diabetes_012']
x = diabetes_data_df.drop(columns=['Diabetes_012'])


In [6]:
y

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
253675    0.0
253676    2.0
253677    0.0
253678    0.0
253679    2.0
Name: Diabetes_012, Length: 253680, dtype: float64

In [7]:
x

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [8]:
x.columns

Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [9]:
model = LogisticRegression()
model.fit(x, y)

coefficients_df = pd.DataFrame(model.coef_, columns=['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income']).T

coefficients_df.columns = ['0 (No diabetes)', '1 (Pre-diabetic)', '2 (Diabetic)']

coefficients_df

c:\Users\David\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,0 (No diabetes),1 (Pre-diabetic),2 (Diabetic)
HighBP,-0.397544,0.074882,0.322662
HighChol,-0.352493,0.192110,0.160382
CholCheck,0.569716,-0.141678,-0.428038
BMI,-0.022415,-0.003680,0.026094
Smoker,0.036578,-0.075148,0.038569
Stroke,-0.053902,-0.033068,0.086970
HeartDiseaseorAttack,-0.159029,-0.015379,0.174408
PhysActivity,0.088960,-0.090527,0.001567
Fruits,0.084677,-0.046716,-0.037960
Veggies,0.097283,-0.082826,-0.014457


In [10]:
#Split data into training ad testing datasets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)


In [11]:
x_train.nunique()

HighBP                   2
HighChol                 2
CholCheck                2
BMI                     81
Smoker                   2
Stroke                   2
HeartDiseaseorAttack     2
PhysActivity             2
Fruits                   2
Veggies                  2
HvyAlcoholConsump        2
AnyHealthcare            2
NoDocbcCost              2
GenHlth                  5
MentHlth                31
PhysHlth                31
DiffWalk                 2
Sex                      2
Age                     13
Education                6
Income                   8
dtype: int64

In [12]:
#Logistic Regression Model with Original Data

from sklearn.linear_model import LogisticRegression

logi_model = LogisticRegression(solver='lbfgs', random_state=42)
logi_model.fit(x_train,y_train)


c:\Users\David\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [13]:
#Prediction and Confusion Matrix
prediction = logi_model.predict(x_test)
c_matrix = confusion_matrix(y_test, prediction)
print(c_matrix)

[[52494     0   984]
 [ 1105     0    74]
 [ 7611     0  1152]]


In [14]:
report = classification_report(y_test,prediction)
print(report)
accuracy = accuracy_score(y_test, prediction)
print(f'Model Accuracy: {accuracy:.2f}')


c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

         0.0       0.86      0.98      0.92     53478
         1.0       0.00      0.00      0.00      1179
         2.0       0.52      0.13      0.21      8763

    accuracy                           0.85     63420
   macro avg       0.46      0.37      0.38     63420
weighted avg       0.80      0.85      0.80     63420

Model Accuracy: 0.85


c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
#Saving the Model
joblib.dump(logi_model, '../Resources/diabetes_logi_regress_model.pkl')

['../Resources/diabetes_logi_regress_model.pkl']

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

In [18]:
logi_model.fit(x_train_scaled,y_train)

LogisticRegression(random_state=42)

In [19]:
y_prediction_scaled = logi_model.predict(x_test_scaled)

In [20]:
c_matrix_scaled = confusion_matrix(y_test, y_prediction_scaled)
print(c_matrix_scaled)

[[52203     0  1275]
 [ 1090     0    89]
 [ 7196     0  1567]]


In [21]:
report_scaled = classification_report(y_test,y_prediction_scaled)
print(report_scaled)
accuracy_scaled = accuracy_score(y_test, y_prediction_scaled)
print(f'Model Accuracy: {accuracy_scaled:.2f}')


              precision    recall  f1-score   support

         0.0       0.86      0.98      0.92     53478
         1.0       0.00      0.00      0.00      1179
         2.0       0.53      0.18      0.27      8763

    accuracy                           0.85     63420
   macro avg       0.47      0.38      0.39     63420
weighted avg       0.80      0.85      0.81     63420

Model Accuracy: 0.85


c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
#Data model optimization

In [23]:
hyperparameters = [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

Results = []

for c_value in hyperparameters:
    optimizing_model = LogisticRegression(C=c_value, multi_class='ovr', max_iter=1000)
    optimizing_model.fit(x_train_scaled,y_train)

    y_optimized_prediction = optimizing_model.predict(x_test_scaled)
    accuracy = accuracy_score(y_test, y_optimized_prediction)
    precision = precision_score(y_test, y_optimized_prediction,average='macro')
    recall = recall_score(y_test, y_optimized_prediction,average='macro')

    Results.append({
        'Model Version': f'Logistic Regression (C={c_value})',
        'Hyperparameters': f'C={c_value}',
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall
    })

data_model_optimization_results = pd.DataFrame(Results)

data_model_optimization_results

c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\David\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\David\anaconda3\Lib\site-packag

,Model Version,Hyperparameters,Accuracy,Precision,Recall
0,Logistic Regression (C=1e-07),C=1e-07,0.843236,0.281079,0.333333
1,Logistic Regression (C=1e-06),C=1e-06,0.843236,0.281079,0.333333
2,Logistic Regression (C=1e-05),C=1e-05,0.843220,0.414418,0.333391
3,Logistic Regression (C=0.0001),C=0.0001,0.847698,0.475685,0.364199
4,Logistic Regression (C=0.001),C=0.001,0.847808,0.466414,0.381959
5,Logistic Regression (C=0.01),C=0.01,0.847934,0.466364,0.384617
6,Logistic Regression (C=0.1),C=0.1,0.847871,0.466045,0.384687
7,Logistic Regression (C=1),C=1,0.847856,0.465983,0.384681
8,Logistic Regression (C=10),C=10,0.847856,0.465983,0.384681
9,Logistic Regression (C=100),C=100,0.847856,0.465983,0.384681


In [24]:
data_model_optimization_results.to_csv('../Resources/data_model_optimization_results.csv', index=False)